In [32]:
import numpy as np
import pandas as pd
from clean import manifest_clinical_merge
from collections import Counter

In [2]:
pd.set_option('display.max_columns', 100)

# Data Prep

In [3]:
manifest_df = pd.read_csv('../Manifest/GCD_TARGET_Data_Manifest_AML_NBL_WT_RT.csv')

wt_disc_df = pd.read_excel('../Clinical_Data/TARGET_WT_ClinicalData_Discovery_20160714_public.xlsx')
aml_disc_df = pd.read_excel('../Clinical_Data/TARGET_AML_ClinicalData_20160714.xlsx')
nbl_disc_df = pd.read_excel('../Clinical_Data/TARGET_NBL_ClinicalData_20151124.xlsx')

WT_df = manifest_clinical_merge(manifest_df, wt_disc_df, 'TARGET-WT')
AML_df = manifest_clinical_merge(manifest_df, aml_disc_df, 'TARGET-AML')
NBL_df = manifest_clinical_merge(manifest_df, nbl_disc_df, 'TARGET-NBL')

/Users/david/statsbomb/RNAseq_Cancer_Biomarkers/scripts/clean.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  target_df['TARGET USI'] = target_df.loc[:, 'entity_submitter_id'].apply(lambda x: x[:16])


In [4]:
from clean import assay_transpose

In [5]:
assay_df = pd.read_csv('../TARGET_NBL_AML_RT_WT_TMMCPM_log2_Norm_Counts.csv')
assay_t_df = assay_transpose(assay_df)

In [6]:
from clean import assay_clinical_merge

In [7]:
AML_genes = assay_clinical_merge(assay_t_df, AML_df)
WT_genes = assay_clinical_merge(assay_t_df, WT_df)
NBL_genes = assay_clinical_merge(assay_t_df, NBL_df)

# Modeling on AML_genes

## Classification - predict low_risk / not_low_risk in AML patients

1) Variance comparison: Subset of patients under 03 09A classifcation - choosing 1000 genes of highest/lowest variance 
between labeled low_risk, not_low_risk group

2) Model performance when randomly sampling genes

3) Lasso logistic regression

4) Tree modeling with features considered important by Lasso

In [46]:
from model_comp import data_prep_columns, model_prep, model_comp, model_prep_loc, xgboost_tuner, create_model_df
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
from sklearn.ensemble import  RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier

# 1. Variance Comparison

### High Variance

In [215]:
df, a = data_prep_columns(AML_genes, 'Max')
X_train, X_test, y_train, y_test, holdout, X = model_prep(df, a)
xg1, rf1 = model_comp(X_train, X_test, y_train, y_test)

/Users/david/statsbomb/RNAseq_Cancer_Biomarkers/scripts/model_comp.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data['label'] = y.copy()
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Ensemble Log Loss 0.28771180389607903
Gradient Boost Log Loss 0.36210740308746076
Random Forest Log Loss 0.3262005795045274
XGBoost Log Loss 0.22184269636164647


[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.0s finished


In [216]:
df.head()

,TARGET USI,Diagnostic ID,Gender,Race,Ethnicity,Age at Diagnosis in Days,First Event,Event Free Survival Time in Days,Vital Status,Overall Survival Time in Days,Year of Diagnosis,Year of Last Follow Up,Protocol,WBC at Diagnosis,Bone marrow leukemic blast percentage (%),Peripheral blasts (%),CNS disease,Chloroma,FAB Category,t(6;9),t(8;21),t(3;5)(q25;q34),t(6;11)(q27;q23),t(9;11)(p22;q23),t(10;11)(p11.2;q23),t(11:19)(q23:p13.1),inv(16),del5q,del7q,del9q,monosomy 5,monosomy 7,trisomy 8,trisomy 21,MLL,Minus Y,Minus X,Cytogenetic Code Other,Cytogenetic Complexity,Primary Cytogenetic Code,ISCN,FLT3/ITD positive?,FLT3/ITD allelic ratio,FLT3 PM,NPM mutation,CEBPA mutation,WT1 mutation,c-Kit Mutation Exon 8,c-Kit Mutation Exon 17,MRD at end of course 1,...,ENSG00000280913.1,ENSG00000280916.1,ENSG00000280927.1,ENSG00000280936.1,ENSG00000280953.1,ENSG00000280987.1,ENSG00000281005.1,ENSG00000281021.1,ENSG00000281026.1,ENSG00000281100.1,ENSG00000281103.1,ENSG00000281106.1,ENSG00000281128.1,ENSG00000281131.1,ENSG00000281183.1,ENSG00000281189.1,ENSG00000281195.1,ENSG00000281205.1,ENSG00000281207.1,ENSG00000281332.1,ENSG00000281344.1,ENSG00000281357.1,ENSG00000281358.1,ENSG00000281376.1,ENSG00000281383.1,ENSG00000281392.1,ENSG00000281398.1,ENSG00000281404.1,ENSG00000281406.1,ENSG00000281433.1,ENSG00000281445.1,ENSG00000281468.1,ENSG00000281490.1,ENSG00000281501.1,ENSG00000281527.1,ENSG00000281538.1,ENSG00000281555.1,ENSG00000281571.1,ENSG00000281628.1,ENSG00000281649.1,ENSG00000281691.1,ENSG00000281706.1,ENSG00000281741.1,ENSG00000281789.1,ENSG00000281896.1,ENSG00000281912.1,__no_feature,__ambiguous,__alignment_not_unique,Low Risk
3,TARGET-20-PADZCG,09A,Female,Unknown,Not Hispanic or Latino,5325,Relapse,314,Dead,536,1996,1998,CCG-2961,29.3,58,5,No,No,M4,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,,0,Normal,"46,XX[20]",No,NaN,No,No,No,No,Not done,Not done,Unknown,...,-2.42284,0.314777,1.74517,-6.92714,-6.12136,1.40683,0.995511,-0.223406,2.2914,-0.613341,3.9011,1.61224,-4.6821,-6.92714,0.8846,-2.4712,0.732776,0.678398,0.391151,1.82242,1.4786,1.37983,0.894175,-1.11056,-0.330849,0.0915962,1.95643,-0.547052,-6.92714,-1.45498,1.12314,-2.93057,2.9388,1.76878,0.580973,-0.832714,-1.78095,0.132623,-0.682824,5.46561,-0.330849,-1.84323,-0.801464,-0.364707,2.1646,-0.152689,17.1243,14.6061,17.8527,0
5,TARGET-20-PAEAKL,09A,Female,Black or African American,Not Hispanic or Latino,5218,Relapse,134,Dead,163,1997,1997,CCG-2961,96.1,80,81,Yes,No,M5,No,No,No,No,Yes,No,Yes,No,No,No,No,No,No,No,No,Yes,No,No,,1,MLL,"46,XX,t(11:19)(q23:p13.1)[20]",No,NaN,No,No,No,No,Not done,Not done,Unknown,...,-5.80459,-6.92714,1.03526,-6.92714,-6.92714,0.113774,1.32433,-4.41384,0.664587,-6.92714,1.21435,-6.92714,-6.92714,-5.80459,-0.0939682,-2.53384,-1.48807,-4.14335,-1.05214,0.929309,0.340124,0.95113,1.78753,-0.202203,6.32096,-0.544956,1.04881,0.496696,-6.92714,-2.79931,-0.914236,-1.94482,1.12118,3.16766,-2.61702,-1.89207,-3.1249,1.05554,-1.94482,3.83862,-2.17756,-0.717812,-0.88817,1.75482,0.456577,0.965496,16.7942,15.3635,19.6702,0
9,TARGET-20-PAEERJ,09A,Female,White,Not Hispanic or Latino,5082,Relapse,259,Dead,373,1997,1998,CCG-2961,71.9,60,41,No,No,M4,No,No,Yes,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,,1,Other,"46,XX,t(3:5)(q23:q33)[19]/46,XX[1]",Yes,0.55,No,No,No,Yes,Not done,Not done,Unknown,...,-3.82289,-3.00939,2.17987,-6.92714,-6.92714,1.94937,-1.00327,0.722174,3.02575,-3.82289,4.43513,-3.49809,-6.92714,-6.92714,1.32028,-4.50961,1.67087,-2.35434,-0.680935,1.47645,1.32543,0.512625,0.884345,-0.488087,-1.00327,-1.25643,2.91237,-0.209853,-6.92714,-1.35175,-0.0681887,-2.81572,-0.0817453,0.557041,0.30953,-2.56667,-1.85807,-0.470147,1.27314,6.07709,0.768241,-0.600677,-1.56366,-0.506252,2.3282,-2.90931,16.916,14.5652,17.571,0
10,TARGET-20-PAEFGR,09A,Female,White,Not Hispanic or Latino,549,Relapse,497,Alive,4022,1997,2008,CCG-2961,51.2,75,41,No,No,M2,No,No,No,No,No,No,No,Yes,No,No,No,No,No,No,No,No,No,No,,1,inv(16),"46,XX,inv(16)(p13q22)[21]/46,XX[1]",No,NaN,No,No,No

In [218]:
xg1_important = X.columns[np.array(xg1.feature_importances_) > 0]
rf1_important = X.columns[np.array(rf1.feature_importances_) > 0]

In [219]:
print(len(xg1_important))
len(rf1_important)

205


697

### country frequency of genes being labeled 'important' by RF or XGboost

In [35]:
# xg_gene_counter = Counter()
# rf_gene_counter = Counter()

In [36]:
# for gene in xg1_important:
#     xg_gene_counter[gene] +=1
# for gene in rf1_important:
#     rf_gene_counter[gene] +=1

### 1b. low variance genes between two labeled groups

In [220]:
df, a = data_prep_columns(AML_genes, 'Min')
X_train, X_test, y_train, y_test, holdout, X = model_prep(df, a)
xg2, rf2 = model_comp(X_train, X_test, y_train, y_test)

/Users/david/statsbomb/RNAseq_Cancer_Biomarkers/scripts/model_comp.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data['label'] = y.copy()
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Ensemble Log Loss 0.4236865689681798
Gradient Boost Log Loss 0.49256451159202913
Random Forest Log Loss 0.48223471620602953
XGBoost Log Loss 0.3429010650394736


[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.0s finished


In [221]:
xg2_important = X.columns[np.array(xg2.feature_importances_) > 0]
rf2_important = X.columns[np.array(rf2.feature_importances_) > 0]

In [223]:
print(len(xg2_important))
len(rf2_important)

208


766

In [222]:
# for gene in xg2_important:
#     xg_gene_counter[gene] +=1
# for gene in rf2_important:
#     rf_gene_counter[gene] +=1

# 2.random gene selection

- random_cols_ are **column index numbers** chosen at random

- does not perform as well as the variation selected columns

- use **model_prep** instead of **model_prep_loc** as random_columns_ is a randomly generated list of numbers 
used to select columns

In [230]:
random_cols = np.random.choice(21404, 2000)
random_set = set(random_cols)
random_cols = list(random_set)
random_cols_ = [x + 84 for x in random_cols]

In [231]:
# random_cols_ are column index numbers

In [232]:
len(random_cols_)

1914

In [233]:
df, __ = data_prep_columns(AML_genes, 'neither')
X_train, X_test, y_train, y_test, holdout, X = model_prep(df, random_cols_)
xg_random, rf_random = model_comp(X_train, X_test, y_train, y_test)

/Users/david/statsbomb/RNAseq_Cancer_Biomarkers/scripts/model_comp.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data['label'] = y.copy()
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Ensemble Log Loss 0.39491046043536376
Gradient Boost Log Loss 0.4643323954434488
Random Forest Log Loss 0.4528452946267843
XGBoost Log Loss 0.3226523800073443


[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.0s finished


In [228]:
random_cols = np.random.choice(21404, 2000)
random_set = set(random_cols)
random_cols = list(random_set)
random_cols_ = [x + 84 for x in random_cols]
df, __ = data_prep_columns(AML_genes, 'neither')
X_train, X_test, y_train, y_test, holdout, X = model_prep(df, random_cols_)
xg_random2, rf_random2 = model_comp(X_train, X_test, y_train, y_test)

/Users/david/statsbomb/RNAseq_Cancer_Biomarkers/scripts/model_comp.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data['label'] = y.copy()
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Ensemble Log Loss 0.3857378566561362
Gradient Boost Log Loss 0.450060344474042
Random Forest Log Loss 0.42749734747068563
XGBoost Log Loss 0.3699347494119728


[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.0s finished


In [234]:
random_cols = np.random.choice(21404, 2000)
random_set = set(random_cols)
random_cols = list(random_set)
random_cols_ = [x + 84 for x in random_cols]
df, __ = data_prep_columns(AML_genes, 'neither')
X_train, X_test, y_train, y_test, holdout, X = model_prep(df, random_cols_)
xg_random3, rf_rando3 = model_comp(X_train, X_test, y_train, y_test)

/Users/david/statsbomb/RNAseq_Cancer_Biomarkers/scripts/model_comp.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data['label'] = y.copy()
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Ensemble Log Loss 0.3772384943194
Gradient Boost Log Loss 0.504297547416616
Random Forest Log Loss 0.4216150371839307
XGBoost Log Loss 0.3159291592955187


[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.0s finished


# 3. Lasso Logistic Regression using all columns

- lasso reduces to 3212 genes instead of 20,000+

In [69]:
from sklearn.linear_model import LogisticRegression

In [235]:
lasso_columns = df.iloc[:, 84:-4].columns
df, __ = data_prep_columns(AML_genes, 'neither')
X_train, X_test, y_train, y_test, holdout, X = model_prep_loc(df, lasso_columns)
log_model = LogisticRegression(penalty='l1', solver='saga')
log_model.fit(X_train, y_train)
y_pred = log_model.predict_proba(X_test)
log_loss(y_test, y_pred)

/Users/david/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


0.21263654943160015

In [236]:
log_coefs = np.array(log_model.coef_)
mask = log_coefs > 0
important_lasso_genes = lasso_columns[mask[0]]

In [237]:
len(important_lasso_genes)

3110

In [ ]:
# for gene in important_lasso_genes:
#     xg_gene_counter[gene] +=1
#     rf_gene_counter[gene] +=1

### 4. Trees Performance Using Lasso Genes

In [238]:
df, __ = data_prep_columns(AML_genes, 'neither')
X_train, X_test, y_train, y_test, holdout, X = model_prep_loc(df, important_lasso_genes)
xg_lass, rf_lass = model_comp(X_train, X_test, y_train, y_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Ensemble Log Loss 0.38889814893691244
Gradient Boost Log Loss 0.4288557996806494
Random Forest Log Loss 0.40714678601797455
XGBoost Log Loss 0.38903383744528164


[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.0s finished


In [239]:
xg_lass_important = X.columns[np.array(xg_lass.feature_importances_) > 0]
rf_lass_important = X.columns[np.array(rf_lass.feature_importances_) > 0]

In [86]:
xgboost_tuner(X_train, X_test, y_train, y_test, [4, 6, 8, 10, 12])

XGB n_estimators: 400 log loss 0.4201527334044914
XGB n_estimators: 600 log loss 0.3856683401894328
XGB n_estimators: 800 log loss 0.3854842494317406
XGB n_estimators: 1000 log loss 0.3859489940459261
XGB n_estimators: 1200 log loss 0.3860728672479053


# 5. Feature Importance When Running Tree Over All Features

In [240]:
all_columns = df.iloc[:, 84:-4].columns
df, __ = data_prep_columns(AML_genes, 'neither')
X_train, X_test, y_train, y_test, holdout, X = model_prep_loc(df, all_columns)
xg_all, rf_all = model_comp(X_train, X_test, y_train, y_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Ensemble Log Loss 0.3417890185477423
Gradient Boost Log Loss 0.4407397864037863
Random Forest Log Loss 0.39492259532890717
XGBoost Log Loss 0.25292958437490304


[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.0s finished


In [241]:
xg_all_important = X.columns[np.array(xg_all.feature_importances_) > 0]
rf_all_important = X.columns[np.array(rf_all.feature_importances_) > 0]

In [242]:
print(len(xg_all_important))
len(rf_all_important)

332


1147

# Comparing the features that the models think are important

### Lasso vs. XGBoost vs. RF (on all columns)

In [243]:
print(len(important_lasso_genes))
print(len(xg_all_important))
len(rf_all_important)

3110
332


1147

In [244]:
set_lasso = set(important_lasso_genes)
set_xg = set(xg_all_important)
set_rf = set(rf_all_important)

In [245]:
print(len(set(important_lasso_genes)))
print(len(set(xg_all_important)))
len(set(rf_all_important))

3110
332


1147

Genes deemed important by both **xg and rf**

In [101]:
xg_rf_intersection = set_xg.intersection(set_rf)
len(xg_rf_intersection)

48

Genes deemed important by both **lasso and rf**

In [102]:
lasso_rf_intersection = set_lasso.intersection(set_rf)
len(lasso_rf_intersection)

293

Genes deemed important by both **lasso and xg**

In [103]:
lasso_xg_intersection = set_lasso.intersection(set_xg)
len(lasso_xg_intersection)

76

Intersection of ** lasso, xg and rf**

In [104]:
three_way = set.intersection(set_lasso, set_xg, set_rf)
len(three_way)

11

In [105]:
three_way

{'ENSG00000099998.16',
 'ENSG00000124491.14',
 'ENSG00000125430.7',
 'ENSG00000130649.8',
 'ENSG00000151468.10',
 'ENSG00000159708.16',
 'ENSG00000163106.9',
 'ENSG00000198842.8',
 'ENSG00000228172.4',
 'ENSG00000260182.1',
 'ENSG00000266733.4'}

### THESE ARE OUR MOST IMPORTANT GENE ENSEMBLES

'ENSG00000099998.16',
 'ENSG00000124491.14',
 'ENSG00000125430.7',
 'ENSG00000130649.8',
 'ENSG00000151468.10',
 'ENSG00000159708.16',
 'ENSG00000163106.9',
 'ENSG00000198842.8',
 'ENSG00000228172.4',
 'ENSG00000260182.1',
 'ENSG00000266733.4'

# Making a table of genes 

In [106]:
all_genes = set.union(set_lasso, set_xg, set_rf)
len(all_genes)

4285

In [ ]:
# test1 = ['a', 'b', 'c']
# test2 = ['d', 'c', 'e']
# test_list ['a', 'b', 'c']
# test_dict = {}
# for letter in test_list:
#     if item in 



In [107]:
# gene_dict = {}
# for gene in all_genes:
#     if gene in three_way:
#         gene_dict[gene] = 'RF, XGBoost, Lasso'
#     elif gene in lasso_xg_intersection:
#         gene_dict[gene] = 'XGBoost, Lasso'
#     elif gene in lasso_rf_intersection:
#         gene_dict[gene] = 'RF, Lasso'
        

SETS IM WORKING WITH:


-  three_way (union of all 11)

- lasso_xg_intersection, xg_rf_intersection, lasso_rf_intersection

- set_lasso, set_xg, set_rf (individual)

In [115]:
gene_dict = {}
for gene in all_genes:
    if gene not in gene_dict:
        gene_dict[gene] = []
        if gene in set_lasso:
            gene_dict[gene].append('lasso')
        if gene in set_xg:
            gene_dict[gene].append('xgboost')
        if gene in set_rf:
            gene_dict[gene].append('random forest')
    else:
        if gene in set_lasso:
            gene_dict[gene].append('lasso')
        if gene in set_xg:
            gene_dict[gene].append('xgboost')
        if gene in set_rf:
            gene_dict[gene].append('random forest')

In [180]:
gene_dict = {}
for gene in all_genes:
    if gene not in gene_dict:
        gene_dict[gene] = ''
        if gene in set_lasso:
            gene_dict[gene] += 'lasso '
        if gene in set_xg:
            gene_dict[gene] += 'xgb '
        if gene in set_rf:
            gene_dict[gene] += 'rf '
    else:
        if gene in set_lasso:
            gene_dict[gene] += ' lasso '
        if gene in set_xg:
            gene_dict[gene] += ' xgb '
        if gene in set_rf:
            gene_dict[gene] += ' rf '

In [181]:
len(gene_dict)

4285

In [182]:
gene_dict['ENSG00000259321.1']

'lasso rf '

In [183]:
d = list(gene_dict.values())

In [184]:
important_genes = pd.DataFrame(data = d, index=gene_dict.keys(), columns = ['Model'])

In [189]:
important_genes.head(5)

,Model
ENSG00000173597.7,lasso
ENSG00000143815.13,rf
ENSG00000110583.11,rf
ENSG00000235776.2,rf
ENSG00000104213.11,lasso


In [ ]:
# if gene in set_lasso:
#             gene_dict[gene] += ' lasso '
#         if gene in set_xg:
#             gene_dict[gene] += ' xgb '
#         if gene in set_rf:
#             gene_dict[gene] += ' rf '

In [186]:
lasso_dict = {}
for gene in all_genes:
    if gene in set_lasso:
        lasso_dict[gene] = 1
    else:
        lasso_dict[gene] = 0

rf_dict = {}
for gene in all_genes:
    if gene in set_rf:
        rf_dict[gene] = 1
    else:
        rf_dict[gene] = 0
        
xgb_dict = {}
for gene in all_genes:
    if gene in set_xg:
        xgb_dict[gene] = 1
    else:
        xgb_dict[gene] = 0

In [190]:
a = list(lasso_dict.values())
b = list(rf_dict.values())
c = list(xgb_dict.values())

In [210]:
pd.DataFrame.from_dict([lasso_dict, rf_dict, xgb_dict])

,ENSG00000000003.13,ENSG00000002586.16,ENSG00000002726.18,ENSG00000002933.6,ENSG00000003056.6,ENSG00000003137.7,ENSG00000003249.12,ENSG00000003436.13,ENSG00000004660.13,ENSG00000004939.12,ENSG00000005073.5,ENSG00000005102.11,ENSG00000005108.14,ENSG00000005194.13,ENSG00000005243.8,ENSG00000005381.7,ENSG00000005471.14,ENSG00000005513.9,ENSG00000005884.16,ENSG00000005893.14,ENSG00000006025.10,ENSG00000006042.10,ENSG00000006047.11,ENSG00000006118.13,ENSG00000006283.16,ENSG00000006459.9,ENSG00000006534.14,ENSG00000006611.14,ENSG00000006638.10,ENSG00000006747.13,ENSG00000007001.11,ENSG00000007038.9,ENSG00000007129.16,ENSG00000007237.17,ENSG00000007314.10,ENSG00000007350.15,ENSG00000007402.10,ENSG00000007516.12,ENSG00000007866.17,ENSG00000007944.13,ENSG00000008056.11,ENSG00000008283.14,ENSG00000008311.13,ENSG00000008394.11,ENSG00000008516.15,ENSG00000008517.15,ENSG00000009790.13,ENSG00000010319.5,ENSG00000010704.17,ENSG00000010818.7,...,ENSG00000279978.1,ENSG00000279989.1,ENSG00000280008.1,ENSG00000280011.1,ENSG00000280022.1,ENSG00000280065.1,ENSG00000280079.1,ENSG00000280081.2,ENSG00000280099.1,ENSG00000280123.1,ENSG00000280129.1,ENSG00000280132.1,ENSG00000280153.1,ENSG00000280157.1,ENSG00000280184.1,ENSG00000280187.1,ENSG00000280194.1,ENSG00000280195.1,ENSG00000280211.1,ENSG00000280213.1,ENSG00000280216.1,ENSG00000280237.1,ENSG00000280254.1,ENSG00000280287.1,ENSG00000280303.2,ENSG00000280332.1,ENSG00000280351.2,ENSG00000280401.1,ENSG00000280550.1,ENSG00000280560.1,ENSG00000280594.1,ENSG00000280614.1,ENSG00000280623.1,ENSG00000280649.1,ENSG00000280670.1,ENSG00000280721.1,ENSG00000280777.1,ENSG00000280798.1,ENSG00000280953.1,ENSG00000281106.1,ENSG00000281207.1,ENSG00000281344.1,ENSG00000281398.1,ENSG00000281404.1,ENSG00000281433.1,ENSG00000281468.1,ENSG00000281501.1,ENSG00000281527.1,ENSG00000281571.1,ENSG00000281741.1
0,0,1,1,1,0,1,1,1,0,0,0,1,1,0,1,1,0,1,1,0,0,0,1,1,1,1,1,1,1,0,1,1,1,1,1,0,1,1,1,0,1,1,1,1,1,1,1,0,1,1,...,0,1,1,0,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,0,1,0,1,1,1,1,0,1,1,1,1,1,0,1,1,1,1,1,0,0,0,1,1,1,1,1,1,1
1,1,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,1,0,1,0,0,1,0,0,0,0,1,0,0,...,1,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
